In [5]:
import os, pickle
import numpy as np
from sklearn.cluster import KMeans
from hidden_markov_model import *

def train():
    ''' modify this part accordingly '''
    TRAIN = True
    train_path = 'sana'
    if TRAIN:
        N = 15 # num of hidden states
        M = 20 # num of observation classes
        # initialization
        PI = np.ones(N)/N
        # transition
        A = np.random.uniform(low=0.1, high=1, size=(N, N))
        A = np.tril(A, k=3) # take lower triangle, allow model to stay or go right
        A /= np.sum(A, axis=0)
        # emission
        B = np.random.uniform(low=0.1, high=1, size=(M, N))
        B /= np.sum(B, axis=0)
        # hmm training params
        max_iter = 100
        tol = 0.1

        # feature extraction and clustering
        data, gestures = load_data(train_path)
        # TODO: extract more features, also tune M to prevent too much -inf in B and P ?
        k_means(data, M)
        observations = cluster_data(data)

        # train hmm
        hmm_train(A, B, PI, max_iter, tol, observations, gestures)

def load_data(path):
    data = {}
    gestures = []
    for file in os.listdir(path):
        filename = os.path.splitext(file)[0]
        gesture = filename.split('_')[0]

        # save gesture classes
        if gesture not in gestures:
            gestures.append(gesture)

        # load sensor data
        raw_data = np.loadtxt(os.path.join(path, file))
        print(raw_data)

        data[filename] = extract_features(raw_data) # ts not used
        #print(data.items())

    return data, gestures

def extract_features(data):
    T = data.shape[0]
    print(T)
    features = np.zeros((T, 7))
    features[:, :7] = data[:,:]
    print(features)
    # features[:, 0] = np.linalg.norm(data[:, [1, 2, 3]], axis=1)  # norm
    # features[:, 1] = np.arctan2(data[:, 2], data[:, 1])  # angle
    return features

def k_means(data, K):
    # K-means on train data
    all_data = np.array([])
    for idx, key in enumerate(data):
        if idx == 0:
            all_data = data[key]
        all_data = np.vstack((all_data,data[key]))
    kmeans = KMeans(n_clusters=K).fit(all_data)
    pickle.dump(kmeans, open("k_means.p", "wb"))
    return

def cluster_data(data):
    kmeans = pickle.load(open("k_means.p", "rb"))
    # cluster observation classes
    for key, value in data.items():
        data[key] = kmeans.predict(value)
    return data



In [6]:
#%%
import cv2 
import numpy as np
import os
import time
import math

import threading

#import gesturecnn as myNN

minValue = 70

x0 = 400
y0 = 200
height = 200
width = 200



lastgesture = -1

kernel = np.ones((15,15),np.uint8)
kernel2 = np.ones((1,1),np.uint8)
skinkernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(5,5))

# Which mask mode to use BinaryMask, SkinMask (True|False) OR BkgrndSubMask ('x' key)
binaryMode = True
bkgrndSubMode = True
mask = 0
bkgrnd = 0
counter = 0
# This parameter controls number of image samples to be taken PER gesture
numOfSamples = 301
gestname = ""
path = ""
mod = 0


def hmm_predict(observations):
    hmm_models = pickle.load(open("hmm_models.p", "rb"))
    gestures = [key for key in hmm_models.keys()]
    filenames = [key for key in observations.keys()]
    print(filenames)
    instances = len(filenames)
    log_likelihood = np.zeros((instances,len(gestures)))
    for j, gesture in enumerate(gestures):
        # extract params
        prior = hmm_models[gesture]['prior']
        transition = hmm_models[gesture]['transition']
        emission = hmm_models[gesture]['emission']

        for i, filename in enumerate(filenames):
            obs = observations[filename]
            _, _, P = forward_backward(obs, transition, emission, prior)
            log_likelihood[i,j] = P

    prediction = [gestures[idx] for idx in np.argmax(log_likelihood, axis=1)]
    log = np.max(log_likelihood, axis=1)
    print('Instances: ', filenames, '\nPrediction: ', prediction, '\nLog Likelihood: ', log)

    # test accuracy
    correct = [prediction[idx] in filenames[idx] for idx in range(instances)]
    accuracy = np.sum(correct)/instances
    print('Test accuracy: ', accuracy)

#%%
def skinMask(frame, x0, y0, width, height, framecount):
    global guessGesture, visualize, mod, lastgesture, saveImg
    # HSV values
    low_range = np.array([0, 50, 80])
    upper_range = np.array([30, 200, 255])
    
    cv2.rectangle(frame, (x0,y0),(x0+width,y0+height),(0,255,0),1)
    #roi = cv2.UMat(frame[y0:y0+height, x0:x0+width])
    roi = frame[y0:y0+height, x0:x0+width]
    
    hsv = cv2.cvtColor(roi, cv2.COLOR_BGR2HSV)
    
    #Apply skin color range
    mask = cv2.inRange(hsv, low_range, upper_range)
    
    mask = cv2.erode(mask, skinkernel, iterations = 1)
    mask = cv2.dilate(mask, skinkernel, iterations = 1)
    
    #blur
    mask = cv2.GaussianBlur(mask, (15,15), 1)
    
    #bitwise and mask original frame
    res = cv2.bitwise_and(roi, roi, mask = mask)
    # color to grayscale
    res = cv2.cvtColor(res, cv2.COLOR_BGR2GRAY)
    return res



def binaryMask(frame, x0, y0, width, height, framecount):
    global guessGesture, visualize, mod, lastgesture, saveImg
    
    cv2.rectangle(frame, (x0,y0),(x0+width,y0+height),(0,255,0),1)
    #roi = cv2.UMat(frame[y0:y0+height, x0:x0+width])
    roi = frame[y0:y0+height, x0:x0+width]
    
    gray = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)
    blur = cv2.GaussianBlur(gray,(5,5),2)
    
    th3 = cv2.adaptiveThreshold(blur,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C,cv2.THRESH_BINARY_INV,11,2)
    ret, res = cv2.threshold(th3, minValue, 255, cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)
    return res


def bkgrndSubMask(frame, x0, y0, width, height, framecount):
    global guessGesture, takebkgrndSubMask, visualize, mod, bkgrnd, lastgesture, saveImg
        
    cv2.rectangle(frame, (x0,y0),(x0+width,y0+height),(0,255,0),1)
    roi = frame[y0:y0+height, x0:x0+width]
    #roi = cv2.UMat(frame[y0:y0+height, x0:x0+width])
    roi = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)
        
    #Take background image
    
    if takebkgrndSubMask == True:
        bkgrnd = roi
        takebkgrndSubMask = False
        print("Refreshing background image for mask...")		

    
    #Take a diff between roi & bkgrnd image contents
    diff = cv2.absdiff(roi, bkgrnd)

    _, diff = cv2.threshold(diff, 25, 255, cv2.THRESH_BINARY)
        
    mask = cv2.GaussianBlur(diff, (3,3), 5)
    mask = cv2.erode(diff, skinkernel, iterations = 1)
    mask = cv2.dilate(diff, skinkernel, iterations = 1)
    res = cv2.bitwise_and(roi, roi, mask = mask)
    return res
	
	
	
#%%
def Main():
    global guessGesture, visualize, mod, binaryMode, bkgrndSubMode, mask, takebkgrndSubMask, x0, y0, width, height, saveImg, gestname, path
    quietMode = False
    
    font = cv2.FONT_HERSHEY_SIMPLEX
    size = 0.5
    fx = 10
    fy = 350
    fh = 18
    takebkgrndSubMask =False
        
    #Call CNN model loading callback

        
    ## Grab camera input
    cap = cv2.VideoCapture(0)
    cv2.namedWindow('Original', cv2.WINDOW_NORMAL)

    # set rt size as 640x480
    ret = cap.set(3,640)
    ret = cap.set(4,480)

    framecount = 0
    fps = ""
    start = time.time()

    plot = np.zeros((512,512,3), np.uint8)
    d=[]
    n=[]
    
    while(True):
        ret, frame = cap.read()
        max_area = 0
        
        frame = cv2.flip(frame, 3)
        frame = cv2.resize(frame, (640,480))
        
        if ret == True:
            if bkgrndSubMode == True:
                roi1 = bkgrndSubMask(frame, x0, y0, width, height, framecount)
            
            roi2 = binaryMask(frame, x0, y0, width, height, framecount)
          
            roi3 = skinMask(frame, x0, y0, width, height, framecount)
            
            
            framecount = framecount + 1
            end  = time.time()
            timediff = (end - start)
            if( timediff >= 1):
                #timediff = end - start
                fps = 'FPS:%s' %(framecount)
                start = time.time()
                framecount = 0

        
        
        ## If enabled will stop updating the main openCV windows
        ## Way to reduce some processing power :)
        if not quietMode:
            cv2.imshow('Original',frame)
            cv2.imshow('ROI1', roi1)
            cv2.imshow('ROI2', roi2)
            cv2.imshow('ROI3', roi3)
        moments = cv2.moments(roi2)
 
                # Calculate Hu Moments
        huMoments = cv2.HuMoments(moments)
        for i in range(0,7):
            huMoments[i] = -1* math.copysign(1.0, huMoments[i]) * math.log10(abs(huMoments[i]))
        d=[]
        
        for i in range(0,7):
            d.append(str(huMoments[i][0]))
        n.append(d)
        print(n)
        
        #data, _ = load_data(test_path)
        #data=extract_features(d)
        if framecount%5==4:
            f={'punch':n}
            observations = cluster_data(f)
            hmm_predict(observations)
            n=[]
            framecount=0
        

            
        
        ############## Keyboard inputs ##################
        key = cv2.waitKey(5) & 0xff
        
        if key == ord('x'):
                takebkgrndSubMask = True
                bkgrndSubMode = True
                print("BkgrndSubMask filter active")
        ## Use Esc key to close the program
        if key == 27:
            break
        
        ## Use b key to toggle between binary threshold or skinmask based filters
    

        ## Use i,j,k,l to adjust ROI window
        elif key == ord('i'):
            y0 = y0 - 5
        elif key == ord('k'):
            y0 = y0 + 5
        elif key == ord('j'):
            x0 = x0 - 5
        elif key == ord('l'):
            x0 = x0 + 5

        ## Quiet mode to hide gesture window
    
        
        #elif key != 255:
        #    print key

    #Realse & destroy
    cap.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    train()
    Main()



[[  3.10784575   9.51890758  12.98485032  12.0487045   24.90382608
   17.39232136  24.61681378]
 [  3.10857805   9.45519598  12.94776914  12.01907471  24.90767533
   17.62190544  24.53900514]
 [  3.10778732   9.38228947  13.88227854  12.75553598  26.49532755
  -18.02561208  26.10819533]
 [  3.10812586   9.42064975  13.55810788  12.60310133  26.33022968
  -17.92965954  25.69505624]
 [  3.10741975   9.37734483  13.69400449  12.6046275   26.26624014
  -17.92416623  25.77549795]
 [  3.10820998   9.39893206  13.62214423  12.60438575  27.03374793
  -17.82885828  25.71815781]
 [  3.10755949   9.38975643  13.63468177  12.56764868  26.35701107
  -17.93785759  25.67813897]
 [  3.11016584   9.38743963  13.4056857   12.42159139  26.18817141
  -17.85413852  25.33954693]
 [  3.11087982   9.43905247  13.40218663  12.39000146  25.93730767
  -18.09856222  25.29719706]
 [  3.11534126   9.47235117  13.16525794  12.31427375  25.88607786
  -17.7413084   25.05879763]
 [  3.1155171    9.42798461  12.9530795 

  -17.19858802 -25.37628503]]
[[  3.1399344   10.20633267  13.59587899  12.58090411 -26.11837321
  -17.80360882  25.69864643]
 [  3.12315293  10.07200016  13.52102293  12.54293599 -25.61722436
  -17.6029255   25.95088781]
 [  3.1216757    9.97615271  13.33753701  12.47846907 -25.41217851
  -17.47744541  25.86255501]
 [  3.12913576   9.91777532  13.37744728  12.51486459 -25.47877967
  -17.47672367 -26.01350571]
 [  3.12757158   9.69745178  13.06048453  12.26383673 -25.07873804
  -17.15120324 -25.07430926]
 [  3.12388077   9.6734788   13.02973562  12.14598371 -24.99933639
  -17.04144601 -24.80957997]
 [  3.12202709   9.69024792  13.12599043  12.20632339 -25.05597501
  -17.10466453 -24.99437032]
 [  3.12043928   9.69881216  13.09715786  12.18133441 -25.08179834
  -17.09054443 -24.89812656]
 [  3.12007119   9.64441085  13.10847211  12.12965244 -25.10839862
  -17.02709671 -24.79469313]
 [  3.11785145   9.68261795  13.07364753  12.11573401 -25.06884185
  -17.02958801 -24.75670302]
 [  3.1189

C:\Users\ASUS\hidden_markov_model.py:12: RuntimeWarning: divide by zero encountered in log
  A = np.log(trans)
C:\Users\ASUS\hidden_markov_model.py:120: RuntimeWarning: invalid value encountered in subtract
  xi -= logsumexp(xi, axis=2).reshape((N,N,1)) # normalization


ok 	Iteration:  1 	log-likelihood:  -1424.6671901148193 	all:  [-140.48715087 -174.45258369  -94.49174062 -140.47446661 -104.53613722
 -133.84601555 -168.42614911 -118.75771865 -130.46832965 -218.72689815]
ok 	Iteration:  2 	log-likelihood:  -1186.9809762686134 	all:  [-134.18192723 -161.9922215   -82.67347362 -106.8209746   -54.91159451
 -142.18006571 -123.85407038  -83.66344447 -101.84393213 -194.85927211]
ok 	Iteration:  3 	log-likelihood:  -1136.049403506468 	all:  [-132.4830661  -158.58683814  -78.62084756  -88.95668465  -52.31476997
 -141.15419077 -119.5344592   -81.57041521  -98.62301908 -184.20511282]
ok 	Iteration:  4 	log-likelihood:  -1083.2933909675257 	all:  [-128.32344948 -153.16424661  -75.17302055  -80.81259199  -48.42444681
 -137.5555618  -113.81232798  -78.08628558  -92.9313028  -175.01015738]
ok 	Iteration:  5 	log-likelihood:  -1053.1615086306633 	all:  [-124.84571677 -149.34013865  -73.40718047  -78.54135762  -46.33711583
 -134.57408286 -110.38579112  -75.64490556 

ok 	Iteration:  42 	log-likelihood:  -994.06432188891 	all:  [-114.58287995 -139.14484698  -70.03587301  -81.6175727   -40.19303044
 -127.75596624 -102.36206913  -69.22107635  -86.68499716 -162.46600992]
ok 	Iteration:  43 	log-likelihood:  -993.9068973988946 	all:  [-114.52908739 -139.11809455  -70.0311865   -81.6439421   -40.15323276
 -127.73154761 -102.31616066  -69.18789305  -86.78525732 -162.41049547]
ok 	Iteration:  44 	log-likelihood:  -993.7579715770564 	all:  [-114.4788084  -139.09293516  -70.02679752  -81.66719106  -40.11537944
 -127.70587823 -102.2735107   -69.15741912  -86.88162993 -162.35842202]
ok 	Iteration:  45 	log-likelihood:  -993.6176108411537 	all:  [-114.43205258 -139.06922177  -70.02284352  -81.68734678  -40.0794308
 -127.67926897 -102.2339404   -69.12951572  -86.97344993 -162.31054037]
ok 	Iteration:  46 	log-likelihood:  -993.4856354999835 	all:  [-114.38873151 -139.04683418  -70.01938796  -81.70460328  -40.0453476
 -127.65198134 -102.19726649  -69.10403103  -8

...

[['3.118185850828308', '9.264273644939399', '12.296538732534179', '13.44502181453735', '-26.31580463495172', '-18.974922845566507', '-28.78117639646569'], ['3.115246818438081', '9.258377300041316', '12.308159325487043', '13.559134033525005', '-26.50892360900593', '19.269751087655745', '27.065188593905276'], ['3.1155951393429056', '9.259777188690485', '12.272552085870984', '13.675629792236258', '-26.823942801005863', '18.645259939704445', '26.77886400713253'], ['3.114984562999157', '9.262912176155861', '12.255533060573635', '13.470843289877758', '-26.4619243777808', '18.545510369802873', '26.509802325018736'], ['3.115028426763886', '9.288647617236782', '12.268656398365332', '13.753033457858129', '-26.78198953834696', '20.510433673652493', '27.3122751413937'], ['3.113946528124734', '9.308035064324363', '12.262219551111972', '13.59931521896348', '-26.534081477437425', '-19.24364199333636', '27.399487300665893']]
[['3.118185850828308', '9.264273644939399', '12.296538732534179', '13.4450218

[['3.112360319709443', '9.184511230804961', '12.292807459484491', '13.876189027955343', '27.10044988776085', '-18.479592552107235', '-27.122514935868153'], ['3.1115217429172493', '9.171421528105656', '12.210884586961987', '13.843397755315857', '26.989777731884857', '-18.449388509387056', '-27.05760534225104']]
[['3.112360319709443', '9.184511230804961', '12.292807459484491', '13.876189027955343', '27.10044988776085', '-18.479592552107235', '-27.122514935868153'], ['3.1115217429172493', '9.171421528105656', '12.210884586961987', '13.843397755315857', '26.989777731884857', '-18.449388509387056', '-27.05760534225104'], ['3.1131403995005984', '9.18669415560535', '12.250608603699156', '13.474728079665226', '26.338789037085547', '-18.17813451787814', '-27.434553865459964']]
[['3.112360319709443', '9.184511230804961', '12.292807459484491', '13.876189027955343', '27.10044988776085', '-18.479592552107235', '-27.122514935868153'], ['3.1115217429172493', '9.171421528105656', '12.210884586961987',

[['3.113656775723268', '9.215140608570332', '12.272954455384593', '14.16631993845006', '-27.66418119274572', '-18.886008023674627', '-27.456593715574137'], ['3.1125866866247', '9.196153232081556', '12.270828294959074', '13.976599732235844', '-27.153250837241224', '-18.850982089582967', '-27.432741959584572'], ['3.1137934476607874', '9.192783541754565', '12.241275100084284', '13.594381138577223', '-26.75795857510498', '18.531353054315577', '26.596747860149694'], ['3.11026756669992', '9.1852978399855', '12.270526312107863', '14.282205951558595', '27.70326049551167', '-18.88716877537931', '-27.715074329185654'], ['3.1101070676178963', '9.170079577891213', '12.238798123729469', '13.922561737821052', '27.043023736484173', '-18.545448288287076', '-27.39136259308387'], ['3.1106626605406293', '9.219859251673283', '12.232832957866684', '13.936048072410882', '27.03581395201148', '18.603524048593883', '-27.60377795697014'], ['3.112793923032504', '9.284366933936772', '12.280010597239361', '13.7922

[['3.138374620292118', '9.42464707466673', '12.21518497789351', '12.340699597889307', '24.62286932471833', '17.10821596017147', '25.47609066228958'], ['3.129367659672191', '9.26211315954648', '12.212118928949316', '12.627677895544814', '25.068749521472792', '17.26457030344055', '25.56356066734089'], ['3.1283011323089194', '9.17362531957757', '12.221685604546451', '12.829091380720783', '25.529211259549772', '17.42398883486354', '25.483210189281536'], ['3.129180755295867', '9.161383880961305', '12.194077025661676', '12.90757486110372', '25.564021929546932', '17.490694525152282', '25.665576732325626'], ['3.1272959372001696', '9.092126950616267', '12.187292945493926', '13.060771808918604', '25.773875087320125', '17.60712965209786', '25.921328679472698'], ['3.1268827796692498', '9.104429850559475', '12.235433706998064', '13.332455044677298', '26.141704689301726', '17.884827990370848', '26.595700876337578']]
[['3.138374620292118', '9.42464707466673', '12.21518497789351', '12.340699597889307'

[['3.127138995666665', '8.949587332071397', '12.296877678377529', '13.908777418846924', '-27.624212329469433', '18.547206804581577', '-27.02493405752372'], ['3.1273398453328176', '8.873777161039083', '12.30412765931782', '13.4068027770143', '-26.508711154182183', '17.932156133067974', '-26.346477127430763']]
[['3.127138995666665', '8.949587332071397', '12.296877678377529', '13.908777418846924', '-27.624212329469433', '18.547206804581577', '-27.02493405752372'], ['3.1273398453328176', '8.873777161039083', '12.30412765931782', '13.4068027770143', '-26.508711154182183', '17.932156133067974', '-26.346477127430763'], ['3.1283101709157233', '8.899455193022508', '12.321586242675705', '13.225842688142246', '-26.919868425011813', '17.930853624136265', '-26.002714229455634']]
[['3.127138995666665', '8.949587332071397', '12.296877678377529', '13.908777418846924', '-27.624212329469433', '18.547206804581577', '-27.02493405752372'], ['3.1273398453328176', '8.873777161039083', '12.30412765931782', '1

In [8]:
twod_list = [] 

for i in range (0, 10):                               
    new = []                   
    for j in range (0, 10):    
        new.append(j)       
    twod_list.append(new)
dic={'hey':twod_list}

In [9]:
twod_list

[[0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
 [0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
 [0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
 [0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
 [0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
 [0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
 [0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
 [0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
 [0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
 [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]]

In [10]:
new

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

In [11]:
dic

{'hey': [[0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
  [0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
  [0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
  [0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
  [0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
  [0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
  [0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
  [0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
  [0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
  [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]]}

In [3]:
import numpy as np
n=np.zeros((2,2))


In [4]:
n

array([[0., 0.],
       [0., 0.]])

In [5]:
print(n)

[[0. 0.]
 [0. 0.]]


In [6]:
print("a")

a


In [1]:
n=[1,2,3]

In [2]:
n[0]

1

In [3]:
n[1]

2

In [4]:
s=n[1:]

In [5]:
s

[2, 3]